In [1]:
import json
from pathlib import Path
import pandas as pd

**Python code-comment dataset from CodeSearchNet challenge**

In [2]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
!unzip python.zip
!gzip -d python/final/jsonl/test/python_test_0.jsonl.gz

--2020-11-29 21:02:58--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.179.109
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.179.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/zip]
Saving to: ‘python.zip’

python.zip          100%[===================>] 897.32M  16.5MB/s    in 63s     

2020-11-29 21:04:02 (14.2 MB/s) - ‘python.zip’ saved [940909997/940909997]

Archive:  python.zip
   creating: python/
   creating: python/final/
   creating: python/final/jsonl/
   creating: python/final/jsonl/train/
  inflating: python/final/jsonl/train/python_train_9.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_12.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_10.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_0.jsonl.gz  
  inflating: python/final/jsonl/train/python_train_6.jsonl.gz  
  inflating: python/fin

In [3]:
python_files = sorted(Path('python/').glob('**/*.gz'))
python_columns_list = ['repo', 'path', 'url', 'code', 
                'code_tokens', 'docstring', 'docstring_tokens', 
                'language', 'partition']

# Convert downloaded json files into a dataframe
python_data = pd.concat([pd.read_json(file,
                        orient='records',
                        compression='gzip',
                        lines=True)[python_columns_list] for file in python_files],
                        sort=False)

python_data.head(5)

,repo,path,url,code,code_tokens,docstring,docstring_tokens,language,partition
0,ageitgey/face_recognition,examples/face_recognition_knn.py,https://github.com/ageitgey/face_recognition/b...,"def train(train_dir, model_save_path=None, n_n...","[def, train, (, train_dir, ,, model_save_path,...",Trains a k-nearest neighbors classifier for fa...,"[Trains, a, k, -, nearest, neighbors, classifi...",python,train
1,ageitgey/face_recognition,examples/face_recognition_knn.py,https://github.com/ageitgey/face_recognition/b...,"def predict(X_img_path, knn_clf=None, model_pa...","[def, predict, (, X_img_path, ,, knn_clf, =, N...",Recognizes faces in given image using a traine...,"[Recognizes, faces, in, given, image, using, a...",python,train
2,ageitgey/face_recognition,examples/face_recognition_knn.py,https://github.com/ageitgey/face_recognition/b...,"def show_prediction_labels_on_image(img_path, ...","[def, show_prediction_labels_on_image, (, img_...",Shows the face recognition results visually.\n...,"[Shows, the, face, recognition, results, visua...",python,train
3,ageitgey/face_recognition,face_recognition/api.py,https://github.com/ageitgey/face_recognition/b...,"def _rect_to_css(rect):\n """"""\n Convert ...","[def, _rect_to_css, (, rect, ), :, return, rec...",Convert a dlib 'rect' object to a plain tuple ...,"[Convert, a, dlib, rect, object, to, a, plain,...",python,train
4,ageitgey/face_recognition,face_recognition/api.py,https://github.com/ageitgey/face_recognition/b...,"def _trim_css_to_bounds(css, image_shape):\n ...","[def, _trim_css_to_bounds, (, css, ,, image_sh...","Make sure a tuple in (top, right, bottom, left...","[Make, sure, a, tuple, in, (, top, right, bott...",python,train


**Java code-comment dataset from CodeSearchNet challenge**

In [6]:
!rm java.*
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/java.zip
!unzip java.zip
!gzip -d java/final/jsonl/test/java_test_0.jsonl.gz

--2020-11-29 21:12:36--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/java.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.226.147
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.226.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1060569153 (1011M) [application/zip]
Saving to: ‘java.zip’

java.zip            100%[===================>]   1011M  16.6MB/s    in 63s     

2020-11-29 21:13:41 (15.9 MB/s) - ‘java.zip’ saved [1060569153/1060569153]

Archive:  java.zip
replace java/final/jsonl/train/java_train_12.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: java/final/jsonl/train/java_train_12.jsonl.gz  
replace java/final/jsonl/train/java_train_9.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: java/final/jsonl/train/java_train_9.jsonl.gz  
replace java/final/jsonl/train/java_train_3.jsonl.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: java/final/jsonl/train/java_train_3.jsonl.gz  
repla

In [7]:
java_files = sorted(Path('java/').glob('**/*.gz'))
java_columns_list = ['repo', 'path', 'url', 'code', 
                'code_tokens', 'docstring', 'docstring_tokens', 
                'language', 'partition']

# Convert downloaded json files into a dataframe
java_data = pd.concat([pd.read_json(file,
                        orient='records',
                        compression='gzip',
                        lines=True)[java_columns_list] for file in java_files],
                        sort=False)

java_data.head(5)

,repo,path,url,code,code_tokens,docstring,docstring_tokens,language,partition
0,spring-projects/spring-boot,spring-boot-project/spring-boot/src/main/java/...,https://github.com/spring-projects/spring-boot...,protected final void bindIndexed(Configuration...,"[protected, final, void, bindIndexed, (, Confi...",Bind indexed elements to the supplied collecti...,"[Bind, indexed, elements, to, the, supplied, c...",java,train
1,spring-projects/spring-boot,spring-boot-project/spring-boot/src/main/java/...,https://github.com/spring-projects/spring-boot...,public void setServletRegistrationBeans(\n\t\t...,"[public, void, setServletRegistrationBeans, (,...",Set {@link ServletRegistrationBean}s that the ...,"[Set, {]",java,train
2,spring-projects/spring-boot,spring-boot-project/spring-boot/src/main/java/...,https://github.com/spring-projects/spring-boot...,public void addServletRegistrationBeans(\n\t\t...,"[public, void, addServletRegistrationBeans, (,...",Add {@link ServletRegistrationBean}s for the f...,"[Add, {]",java,train
3,spring-projects/spring-boot,spring-boot-project/spring-boot/src/main/java/...,https://github.com/spring-projects/spring-boot...,public void setServletNames(Collection<String>...,"[public, void, setServletNames, (, Collection,...",Set servlet names that the filter will be regi...,"[Set, servlet, names, that, the, filter, will,...",java,train
4,spring-projects/spring-boot,spring-boot-project/spring-boot/src/main/java/...,https://github.com/spring-projects/spring-boot...,public void addServletNames(String... servletN...,"[public, void, addServletNames, (, String, ......",Add servlet names for the filter.\n@param serv...,"[Add, servlet, names, for, the, filter, .]",java,train
